# Strip plot - excess deaths with benchmark comparison

In [1]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
import random
import altair as alt
pd.options.mode.chained_assignment = None

### Create dummy data

In [2]:
random.seed(0)
data = pd.DataFrame({'Month': ['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
                            '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
                            '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01'],
                  'Deaths to mortality O=E': random.sample(range(-15, 20), 12),
                  'Deaths to benchmark mortality O/E': random.sample(range(-15, 20), 12)})
data['Month'] = pd.to_datetime(data['Month'])

In [3]:
data.sample(3)

,Month,Deaths to mortality O=E,Deaths to benchmark mortality O/E
0,2020-01-01,9,-2
8,2020-09-01,10,10
10,2020-11-01,13,2


### Transform data for Altair strip plot

In [4]:
df = pd.DataFrame()
for x in data['Month'].unique():
    temp = data.loc[data['Month']==x]
    indexVal = temp['Deaths to mortality O=E'].values[0]
    compVal = temp['Deaths to benchmark mortality O/E'].values[0]
    if indexVal < 0 and compVal < 0:
        if indexVal < compVal:
            valRange1 = [i for i in range(compVal-1, indexVal-1, -1)]
            df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1, 
                                         'color':['#01adaa' for i in valRange1]}))
            valRange2 = [i for i in range(-1, compVal-1, -1)]
            df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2, 
                                         'color':['#99d7dd' for i in valRange2]}))
        else:
            if indexVal == compVal:
                valRange2 = [i for i in range(-1, indexVal-1, -1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                             'color':['#01adaa' for i in valRange2]}))
            else:
                valRange1 = [i for i in range(indexVal-1, compVal-1, -1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1,
                                             'color':['#99d7dd' for i in valRange1]}))
                valRange2 = [i for i in range(-1, indexVal-1, -1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2, 
                                             'color':['#01adaa' for i in valRange2]}))
    elif indexVal > 0 and compVal > 0:
        if indexVal < compVal:
            valRange1 = [i for i in range(1, indexVal+1)]
            df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1,
                                         'color':['#ff4d00' for i in valRange1]}))
            valRange2 = [i for i in range(indexVal+1, compVal+1)]
            df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                         'color':['#F79d6C' for i in valRange2]}))
        else:
            if indexVal == compVal:
                valRange2 = [i for i in range(1, indexVal+1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                             'color':['#ff4d00' for i in valRange2]}))
            else:
                valRange1 = [i for i in range(1, compVal+1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1,
                                             'color':['#F79d6C' for i in valRange1]}))
                valRange2 = [i for i in range(compVal+1, indexVal+1)]
                df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                             'color':['#ff4d00' for i in valRange2]}))
    elif indexVal >= 0 and compVal <= 0:
        valRange1 = [i for i in range(1, indexVal+1)]
        df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1,
                                     'color':['#ff4d00' for i in valRange1]}))
        valRange2 = [i for i in range(-1, compVal-1, -1)]
        df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                     'color':['#99d7dd' for i in valRange2]}))
    elif indexVal <= 0 and compVal >= 0:
        valRange1 = [i for i in range(-1, indexVal-1, -1)]
        df = df.append(pd.DataFrame({'Month':[x for i in valRange1],'y':valRange1,
                                     'color':['#01adaa' for i in valRange1]}))
        valRange2 = [i for i in range(1, compVal+1)]
        df = df.append(pd.DataFrame({'Month':[x for i in valRange2],'y':valRange2,
                                     'color':['#F79d6C' for i in valRange2]}))
    else:
        df = df.append(pd.DataFrame({'Month':[x],'y':[0], 'color':['']}))

### Example of one month after data transformation

In [5]:
# 9 deaths above O=E, 2 below benchmark's O/E
df.loc[df['Month']=='2020-01-01']

,Month,y,color
0,2020-01-01,1.0,#ff4d00
1,2020-01-01,2.0,#ff4d00
2,2020-01-01,3.0,#ff4d00
3,2020-01-01,4.0,#ff4d00
4,2020-01-01,5.0,#ff4d00
5,2020-01-01,6.0,#ff4d00
6,2020-01-01,7.0,#ff4d00
7,2020-01-01,8.0,#ff4d00
8,2020-01-01,9.0,#ff4d00
0,2020-01-01,-1.0,#99d7dd


In [6]:
moList = df['Month'].drop_duplicates().tolist()
yMin = data[['Deaths to mortality O=E','Deaths to benchmark mortality O/E']].min().min()
yMax = data[['Deaths to mortality O=E','Deaths to benchmark mortality O/E']].max().max()
valuesY = [x for x in range(yMin-2, yMax+1)]

orgTicksPos = alt.Chart(df).mark_tick(thickness=12,width=30,xOffset=35,yOffset=11,orient='horizontal'
                            ).encode(  #separate for different yOffset values - push marks towards 0
                            x= alt.X('Month:T',scale=alt.Scale(domain=(moList[0] ,moList[-1])),
                                        axis=alt.Axis(grid=False, format='%b-%y',tickOffset=40,
                                        domainWidth=0,labelFontSize=16)),
                            y=alt.Y('y', axis=alt.Axis(values=valuesY,labelFontSize=16,title='Excess deaths')),
                            color=alt.Color('color',scale=None)
                            ).transform_filter(alt.datum.y>0)

orgTicksNeg = alt.Chart(df).mark_tick(thickness=12,width=30,xOffset=35,yOffset=-11,orient='horizontal'
                            ).encode(
                            x= alt.X('Month:T',scale=alt.Scale(domain=(moList[0] ,moList[-1])),
                                        axis=alt.Axis(title=None,grid=False, format='%b-%y',tickOffset=40,
                                        domainWidth=0,labelFontSize=16)),
                            y=alt.Y('y', axis=alt.Axis(values=valuesY,labelFontSize=16)),
                            color=alt.Color('color',scale=None)
                            ).transform_filter(alt.datum.y<0)

zero = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='#696969',xOffset=50,size=2).encode(y='y')

alt.layer(orgTicksPos+orgTicksNeg+zero
        ).properties(width=800,height=500, title='Excess deaths by month'
        ).configure(lineBreak='\n'
        ).configure_axis(labelColor='#6e6e6e',labelFont='Arial',grid=False,labelAngle=0,
                         titleFont='Arial',titleFontSize=14,titleColor='#6e6e6e'
        ).configure_view(strokeWidth=0
        ).configure_title(fontSize=20,color='#6e6e6e',font='Arial',anchor='start',offset=25)


alt.LayerChart(...)

<img src="..\assets\images\excessDeathsLegend.png" width="300" height="200" />